# Задание 1

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [4]:
my_list = list(range(5))
my_list = my_list[::-1]
print(my_list)

[4, 3, 2, 1, 0]


In [77]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
from math import sin, cos, log, exp
import math
@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
     match other:
        case Dual(o_value, o_d):
            return Dual(self.value - o_value, self.d - o_d)
        case Number():
            return Dual(self.value- float(other), self.d)
    def __rsub__(self, other: Number) -> "Dual":
            return Dual(float(other) - self.value, -self.d)
    def __pow__(self, other: Number) -> "Dual":
        return Dual(float(math.pow(self.value,other)), other*float(math.pow(self.value,other-1.0)))
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d )/(o_value**2))
            case Number():
                return Dual(self.value/float(other), self.d/float(other))
    def __rtruediv__(self, other: Number) -> "Dual":
        return Dual(float(other)/self.value, - float(other)*self.d/ (self.value**2) )
    def __neg__(self):
        return Dual(-self.value, -self.d)
    def __pos__(self):
        return Dual(self.value, self.d)

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

def Cos(other: Union["Dual", Number]) -> "Dual":
     match other:
        case Dual(o_value, o_d):
            return Dual(cos(o_value), -o_d*sin(o_value))
        case Number():
            return Dual(cos(float(other)), 0)
                         
def Sin(other: Union["Dual", Number]) -> "Dual":
     match other:
        case Dual(o_value, o_d):
            return Dual(sin(o_value), o_d*cos(o_value))
        case Number():
            return Dual(sin(float(other)), 0)
                            

def Exp(other: Union["Dual", Number]) -> "Dual":
     match other:
        case Dual(o_value, o_d):
            return Dual(exp(o_value), o_d*exp(o_value))
        case Number():
            return Dual(exp(float(other)), 0)
        
def Log(base: Number, other: Union["Dual", Number]) -> "Dual":
     match other:
        case Dual(o_value, o_d):
            return Dual(math.log(o_value, base), o_d/(o_value*log(base, math.e)))
        case Number():
            return Dual(exp(float(other)), 0)
                            
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda *args: func(*list(Dual(arg, 1.0) for i, arg in enumerate(args))).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [80]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return Sin(x)
f_diff = diff(f)
# значение производной в точке x = 2
f_diff(2)

-0.4161468365471424

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [7]:
!which python


"which" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [8]:
# ваш код

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [9]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [10]:
# ваш код

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=math.cos(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)
print(expr)
func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [11]:
print(4)

4


In [12]:
import ast
import random

rate = 1.05
start = 0

BinOp = [ast.Mult(), ast.Add(), ast.Sub(), ast.Div(), ast.Pow()]
Custom_operations = [lambda: ast.Call(func=ast.Name(id='Sin', ctx=ast.Load()), args=[generate_expression()],\
                                      keywords=[]), lambda: ast.Call(func=ast.Name(id='Exp', ctx=ast.Load()), args=[generate_expression()], keywords=[]),lambda: ast.Call(func=ast.Name(id='Cos', ctx=ast.Load()), args=[generate_expression()], keywords=[])]
UnOp = [ ast.USub(), ast.UAdd()]
Variable = ast.Name(id='x', ctx=ast.Load())
Constant = lambda a: ast.Constant(random.randint(a,5))

def generate_unary_operation():
    return random.choice(UnOp)

def generate_binary_operation():
    return random.choice(BinOp)

def generate_custom_operation():
    return random.choice(Custom_operations)()

operation = [lambda: ast.BinOp(
                left=generate_expression(),
                op=generate_binary_operation(),
                right=generate_expression()), lambda: ast.UnaryOp(generate_unary_operation(), generate_expression()),lambda: generate_custom_operation()]


def generate_ast():
    return ast.Expression(body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),body=generate_expression()))

def generate_expression():
    global start
    op = random.randint(0,9)
    if op < start:
        return Variable
    elif op <2*start:
        return Constant(-5)
    start = 2
    return random.choice(operation)()


print(4)

expr = generate_ast()

ast.fix_missing_locations(expr)
print(ast.dump(expr))
func = eval(compile(expr, filename="", mode="eval"))
f_diff = diff(func)

print(f_diff(2))

4
Expression(body=Lambda(args=arguments(posonlyargs=[], args=[arg(arg='x')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=BinOp(left=Constant(value=-5), op=Div(), right=Name(id='x', ctx=Load()))))
1.25


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [93]:
import ast
import random

rate = 1.05
start = 0

BinOp = [ast.Mult(), ast.Add(), ast.Sub(), ast.Div(), ast.Pow()]
Custom_operations = [lambda: ast.Call(func=ast.Name(id='Sin', ctx=ast.Load()), args=[generate_expression()],\
                                      keywords=[]), lambda: ast.Call(func=ast.Name(id='Exp', ctx=ast.Load()), args=[generate_expression()], keywords=[]),lambda: ast.Call(func=ast.Name(id='Cos', ctx=ast.Load()), args=[generate_expression()], keywords=[])]
UnOp = [ ast.USub(), ast.UAdd()]
Variable = list(map(lambda x: ast.Name(id=x, ctx=ast.Load()), "abcdefghijklmnopqrstuvwxyz"))
Constant = lambda a: ast.Constant(random.randint(a,5))
def generate_unary_operation():
    return random.choice(UnOp)

def generate_binary_operation():
    return random.choice(BinOp)

def generate_custom_operation():
    return random.choice(Custom_operations)()

operation = [lambda: ast.BinOp(
                left=generate_expression(),
                op=generate_binary_operation(),
                right=generate_expression()), lambda: ast.UnaryOp(generate_unary_operation(), generate_expression()),lambda: generate_custom_operation()]


def generate_ast(N):
    global variables, Variable
    variables = random.sample("abcdefghijklmnopqrstuvwxyz", k=N)
    Variable = list(map(lambda x: ast.Name(id=x, ctx=ast.Load()), variables))
    return ast.Expression(body=ast.Lambda(
        
        args=ast.arguments(
            args=list(map(lambda x: ast.arg(arg=x),variables))
            ,
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),body=generate_expression()))

def generate_expression():
    global start, Variable
    op = random.randint(0,9)
    if op < start:
        return random.choice(Variable)
    elif op <2*start:
        return Constant(-5)
    start = 2
    return random.choice(operation)()


print(4)

expr = generate_ast(1)
ast.fix_missing_locations(expr)
print(ast.dump(expr))
func = eval(compile(expr, filename="", mode="eval"))

f_diff = diff(func)

print(f_diff(2))

4
Expression(body=Lambda(args=arguments(posonlyargs=[], args=[arg(arg='u')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=BinOp(left=Call(func=Name(id='Exp', ctx=Load()), args=[Constant(value=0)], keywords=[]), op=Mult(), right=Constant(value=-1))))
-0.0


In [90]:
expr = ast.parse("lambda x, y: sin(x)*y", mode="eval")
print(ast.dump(expr))
func = eval(compile(expr, filename="", mode="eval"))
f_diff = diff(func)

print(f_diff(2,1))

Expression(body=Lambda(args=arguments(posonlyargs=[], args=[arg(arg='x'), arg(arg='y')], kwonlyargs=[], kw_defaults=[], defaults=[]), body=BinOp(left=Call(func=Name(id='sin', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]), op=Mult(), right=Name(id='y', ctx=Load()))))


TypeError: must be real number, not Dual

In [15]:
variables = random.choices("abcdefghijklmnopqrstuvwxyz", k=3)
variables

['i', 'z', 'a']

In [64]:
def d(*args, h=1e-5):
    return (f(args[0]+h, args[1]+h)-f(args[0],args[1]))/h
print(d(3,1))

Dual(value=-0.8488830940595048, d=0.0)
